A modified script 

In [1]:
import os
os.environ['MLXLM_USE_MODELSCOPE'] = 'True'
from mlx_lm import load, generate
model, tokenizer = load("mlx-community/DeepSeek-R1-Distill-Qwen-1.5B-bf16")

2025-03-14 14:03:04,904 - modelscope - INFO - Target directory already exists, skipping creation.


In [ ]:
from datasets import load_dataset
folder="/Users/zihuiouyang/Documents/data/" 
system_message = """You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.
SCHEMA:
{schema}"""
def create_conversation(sample):
  return {
    "messages": [
      {"role": "system", "content": system_message.format(schema=sample["context"])},
      {"role": "user", "content": sample["question"]},
      {"role": "assistant", "content": sample["answer"]}
    ]
  }
dataset = load_dataset("b-mc2/sql-create-context", split="train")
dataset = dataset.map(create_conversation, remove_columns=dataset.features,batched=False)
dataset = dataset.train_test_split(test_size=50/150)
dataset_test_valid = dataset['test'].train_test_split(0.5)
print(dataset["train"][45]["messages"])
dataset["train"].to_json(folder + "train.jsonl", orient="records")
dataset_test_valid["train"].to_json(folder + "test.jsonl", orient="records")
dataset_test_valid["test"].to_json(folder + "valid.jsonl", orient="records")


In [2]:
from pathlib import Path
import json
from mlx_lm.tuner import train, evaluate, TrainingArgs
adapter_path = Path("adapters")
adapter_path.mkdir(parents=True, exist_ok=True)

lora_config = {
 "num_layers": 8,
 "lora_parameters": {
    "rank": 8,
    "scale": 20.0,
    "dropout": 0.0,
}}

# Save the LoRA config to the adapter path
with open(adapter_path / "adapter_config.json", "w") as fid:
    json.dump(lora_config, fid, indent=4)    

training_args = TrainingArgs(
    adapter_file=adapter_path / "adapters.safetensors",
    iters=13100,
    steps_per_report=300,
    steps_per_eval=500,
    steps_per_save=1000
)

In [3]:
from mlx.utils import tree_flatten
from mlx_lm.tuner import linear_to_lora_layers
model.freeze()

# Convert linear layers to lora layers
linear_to_lora_layers(model, lora_config["num_layers"], lora_config["lora_parameters"])

num_train_params = (
    sum(v.size for _, v in tree_flatten(model.trainable_parameters()))
)
print(f"Number of trainable parameters: {num_train_params}")

Number of trainable parameters: 311296


In [4]:
with open("/Users/zihuiouyang/Documents/data/train.jsonl", "r") as fid:
        train_data = [json.loads(line) for line in fid]

In [5]:
with open("/Users/zihuiouyang/Documents/data/valid.jsonl", "r") as fid:
        valid_data = [json.loads(line) for line in fid]

In [6]:
from mlx_lm.tuner.datasets import ChatDataset
def make_dataset(ds):
    return ChatDataset(
        ds,
        tokenizer
    )
train_set, valid_set = make_dataset(train_data), make_dataset(valid_data)

In [7]:
import mlx.optimizers as optim

In [8]:
# Put the model in training mode:
model.train()

# Make the optimizer:
opt = optim.Adam(learning_rate=1e-5)

# Make a class to record the training stats:
class Metrics:
    train_losses = []
    val_losses = []
    def on_train_loss_report(self, info):
        self.train_losses.append((info["iteration"], info["train_loss"]))
    def on_val_loss_report(self, info):
        self.val_losses.append((info["iteration"], info["val_loss"]))

metrics = Metrics()

In [9]:
train(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    optimizer=opt,
    train_dataset=train_set,
    val_dataset=valid_set,
    training_callback=metrics,
)

Starting training..., iters: 13100
Iter 1: Val loss 4.576, Val took 11.159s
Iter 300: Train loss 2.092, Learning Rate 1.000e-05, It/sec 1.555, Tokens/sec 611.775, Trained Tokens 118055, Peak mem 7.269 GB
Iter 500: Val loss 1.405, Val took 10.397s
Iter 600: Train loss 1.495, Learning Rate 1.000e-05, It/sec 1.586, Tokens/sec 616.086, Trained Tokens 234573, Peak mem 7.269 GB
Iter 900: Train loss 1.417, Learning Rate 1.000e-05, It/sec 1.582, Tokens/sec 609.713, Trained Tokens 350199, Peak mem 7.269 GB
Iter 1000: Val loss 1.358, Val took 10.764s
Iter 1000: Saved adapter weights to adapters/adapters.safetensors and adapters/0001000_adapters.safetensors.
Iter 1200: Train loss 1.368, Learning Rate 1.000e-05, It/sec 1.597, Tokens/sec 615.101, Trained Tokens 465775, Peak mem 7.269 GB
Iter 1500: Val loss 1.416, Val took 11.091s
Iter 1500: Train loss 1.339, Learning Rate 1.000e-05, It/sec 1.585, Tokens/sec 616.100, Trained Tokens 582362, Peak mem 7.269 GB
Iter 1800: Train loss 1.337, Learning Rate

In [11]:
!mlx_lm.fuse --model /Users/zihuiouyang/.cache/modelscope/hub/models/mlx-community/DeepSeek-R1-Distill-Qwen-1.5B-bf16

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading pretrained model


In [1]:
!mlx_lm.generate --max-tokens 2048 --model /Users/zihuiouyang/.cache/modelscope/hub/models/mlx-community/DeepSeek-R1-Distill-Qwen-1.5B-bf16 --prompt "You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.\nSCHEMA:\nCREATE TABLE table_name_62 (score VARCHAR, opponents VARCHAR)\nUser:Tell me the score of misa eguchi eri hozumi\nAssistant:"

Okay, so I need to translate the user's query into a SQL query based on the provided schema. The user asked, "Tell me the score of misa eguchi eri hozumi." First, I should figure out what the user is asking for. It seems like they want the score of a specific player named "misa eguchi eri hozumi." 

 at the schema, there's a table named table_name_62 with two columns: score and opponents. The user is asking for the score, so I need to select the score column. The name of the player is a bit unusual, but I can assume it's a single name. 

I should make sure the SQL query selects the score from the table. The correct syntax would be SELECT score FROM table_name_62. Since the user didn't specify the table name, I'll leave it as table_name_62. 

't think the opponents column is needed here because the user only asked for the score. So, the final SQL query should just select the score from the table. I'll write it out clearly so the user understands exactly what they're getting.
</think>

 